In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torch.utils.data import DataLoader

In [3]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to your dataset
data_dir = r"C:\Users\golla\Desktop\BTECH\8THSEM"

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [4]:
# Load datasets
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
valid_dataset = datasets.ImageFolder(root=f"{data_dir}/valid", transform=transform)
test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


In [5]:
# Load pre-trained ResNet-50
model = models.resnet50(pretrained=True)

# Modify last layer for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Fake (1) or Real (0)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


C:\Users\golla\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\golla\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Function to calculate accuracy
def calculate_accuracy(loader, model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [7]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    valid_accuracy = calculate_accuracy(valid_loader, model)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Validation Accuracy: {valid_accuracy:.2f}%")

Epoch [1/5], Loss: 0.3995, Validation Accuracy: 81.03%
Epoch [2/5], Loss: 0.2852, Validation Accuracy: 81.03%
Epoch [3/5], Loss: 0.1997, Validation Accuracy: 91.95%
Epoch [4/5], Loss: 0.1183, Validation Accuracy: 93.68%
Epoch [5/5], Loss: 0.0867, Validation Accuracy: 93.10%


In [8]:
# Save the trained model
torch.save(model.state_dict(), "resnet50_fake_news.pth")
print("Model saved successfully!")

# Final Test Accuracy
test_accuracy = calculate_accuracy(test_loader, model)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Model saved successfully!
Test Accuracy: 90.12%


In [21]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load trained model
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # Binary classification
model.load_state_dict(torch.load("resnet50_fake_news.pth", map_location=device))
model = model.to(device)
model.eval()

# Image preprocessing function
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image.to(device)

# Function to test an image
def test_image(image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()
    
    labels = ["Fake", "Real"]
    return labels[predicted_class]

# Example usage
image_path = r"C:\Users\golla\Desktop\fake1.jpg"
prediction = test_image(image_path)
print(f"Prediction: {prediction}")


Prediction: Fake


In [17]:
from torchvision import datasets

data_dir = r"C:\Users\golla\Desktop\BTECH\8THSEM\train"
train_dataset = datasets.ImageFolder(root=data_dir)

print(train_dataset.class_to_idx)  # Prints: {'fake': 0, 'real': 1} or vice versa


{'fake': 0, 'real': 1}
